In [ ]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import basic
import matplotlib.pyplot as plt

In [ ]:
# head = 'HEAD PRINT FORMAT   0\n'\
# 'HEAD SAVE UNIT   336\n'\
# 'DRAWDOWN PRINT FORMAT   0\n\n'
# head

In [ ]:
# kper = range(1,110)
# step = range(1,25)

# with open('RR_2022/rr_all.oc','w') as wr:
#     wr.write(head)
#     for k in kper:
        
#         for s in step:
#             wr.write(f"period {k} step {s}\n")
#             wr.write("PRINT BUDGET\n")
#             wr.write("SAVE BUDGET\n")
            
#             if (s==12) or (s==24):
#                 wr.write('SAVE HEAD\n')
            
#             wr.write('\n\n')


In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2016')

datestart = info['start_date']
numdays = info['numdays']
name = info['name']

In [ ]:
# Set path to example datafiles
loadpth = os.path.join( "data", "zonbud_examples")
cbc_f = os.path.join(ml.model_ws,'Results', 'RRMF.cbc')

In [ ]:
from flopy.utils import ZoneBudget

In [ ]:
arrays.head()

In [ ]:
arrays = gpd.read_file('GIS/grid.shp')
arrays = arrays.dropna(subset = 'zone')
arrays.head()

In [ ]:
ml = basic.load_model()

In [ ]:
zones = np.zeros((ml.dis.nlay, ml.dis.nrow, ml.dis.ncol), dtype = int)

for lay in range(3):
    zones[lay,arrays.loc[:,'row']-1, arrays.loc[:,'column']-1] =arrays.loc[:,'zone'].astype(int)

plt.imshow(zones[0])

In [ ]:
import conda_scripts.arich_functions as af
import conda_scripts.utils.folium_maps as fm
import conda_scripts.plot_help as ph
import pandas as pd

In [ ]:
aliases = {1:'Mirabel', 2:'Wohler', 3:'Upstream'}

In [ ]:
modelgrid = af.get_model_shp(ml.modelgrid)
zarr_gdf = af.array2rc(zones[0], 'zone')

zarr_gdf = gpd.pd.merge(zarr_gdf, modelgrid.drop(columns = ['row', 'col']), on = ['i','j'])

zarr_gdf = zarr_gdf.query("zone>0")

zarr_gdf.loc[:,'zone_name'] = zarr_gdf.zone.replace(aliases)

zarr_gdf = gpd.GeoDataFrame(zarr_gdf, geometry = 'geometry', crs = 2226)
zarr_gdf = zarr_gdf.dissolve('zone')

fig, ax = basic.map_river()
zarr_gdf.plot('zone_name',ax = ax, alpha = .4)

zarr_gdf.plot('zone_name',ax = ax,  facecolor = 'None', edgecolor = 'k', lw = 3)

ph.label_poly(zarr_gdf, ax = ax,  column = 'zone_name', label_exterior = False, text_color = 'k')

ax.set_title("Model Zones")

plt.savefig(os.path.join('versions/website_info/zone_map.png'), bbox_inches = 'tight', dpi =300)

In [ ]:
z = ZoneBudget(cbc_f, z = zones, aliases =aliases)

In [ ]:
df = pd.DataFrame(z.get_budget())
df.loc[:,'Date'] = pd.to_datetime(datestart) + pd.to_timedelta(df.loc[:,'stress_period'], unit = 'D')+\
pd.to_timedelta(df.loc[:,'stress_period'], unit = 'H')
df = df.set_index(['Date','name']).drop(columns=['totim','time_step','stress_period'])

In [ ]:
dfi = df.loc[:,['Wohler']].groupby(level = [0,1]).sum().unstack().droplevel(0,1)
fig, ax = plt.subplots(2,1, sharex = True, figsize = (10,6))
dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'FROM_').drop(columns ='FROM_STORAGE').plot(ax = ax[0])
ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))

dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'TO_').drop(columns ='TO_STORAGE').plot(ax = ax[1])
ax[1].legend(loc = 'upper left', bbox_to_anchor = (1,1))

In [ ]:
dfi = df.loc[:,['Mirabel']].groupby(level = [0,1]).sum().unstack().droplevel(0,1)
fig, ax = plt.subplots(2,1, sharex = True, figsize = (10,6))
dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'FROM_').drop(columns ='FROM_STORAGE').plot(ax = ax[0])
ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))

dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'TO_').drop(columns ='TO_STORAGE').plot(ax = ax[1])
ax[1].legend(loc = 'upper left', bbox_to_anchor = (1,1))

In [ ]:
dfi = df.loc[:,['Upstream']].groupby(level = [0,1]).sum().unstack().droplevel(0,1)
fig, ax = plt.subplots(2,1, sharex = True, figsize = (10,6))
dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'FROM_').drop(columns ='FROM_STORAGE').plot(ax = ax[0])
ax[0].legend(loc = 'upper left', bbox_to_anchor = (1,1))

dfi.loc[:,dfi.sum().abs()>0].filter(regex = 'TO_').drop(columns ='TO_STORAGE').plot(ax = ax[1])
ax[1].legend(loc = 'upper left', bbox_to_anchor = (1,1))

fig.suptitle('Upstream Zone Area')

In [ ]:
df.index.get_level_values(0).unique()

In [ ]:
# ZoneBudget.write_zone_file('zonedbud.zbarr',zones)